# Imports

In [ ]:
## Import necessary libraries here
import cv2
import numpy as np
from scipy.io import loadmat
from scipy import ndimage
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import copy
import os
import time
%matplotlib inline

# Shape Alignment with the Point-to-point version of Iterative Closest Point Algorithm


- This problem involves aligning two sets of points using a global image transformation and return a transformation that maps non-zero points in $im1$ to non-zero points in $im2$.
- This method is repeated to get better & better transformations.


<img src="https://drive.google.com/uc?id=1qSrXkFvyG6rnzXBqJAYG9uqCOoMYSv3W" width="1000"/>

## Data

**WARNING: Colab deletes all files everytime runtime is disconnected. Make sure to re-download the inputs when it happens.**

In [ ]:
# Download Data -- run this cell only one time per runtime
!gdown 18Px9uQyY1fGGyEAQhzt3h4yDQonU_Sgm
!unzip "/content/part2_images.zip" -d "/content/"

Downloading...
From: https://drive.google.com/uc?id=18Px9uQyY1fGGyEAQhzt3h4yDQonU_Sgm
To: /content/part2_images.zip
100% 78.4k/78.4k [00:00<00:00, 69.5MB/s]
Archive:  /content/part2_images.zip
   creating: /content/part2_images/
 extracting: /content/part2_images/Bone_1.png  
 extracting: /content/part2_images/elephant_1.png  
 extracting: /content/part2_images/brick_2.png  
 extracting: /content/part2_images/Heart_2.png  
 extracting: /content/part2_images/Bone_2.png  
  inflating: /content/part2_images/elephant_2.png  
 extracting: /content/part2_images/brick_1.png  
 extracting: /content/part2_images/Heart_1.png  
  inflating: /content/part2_images/device7_1.png  
  inflating: /content/part2_images/device7_2.png  
 extracting: /content/part2_images/fork_2.png  
 extracting: /content/part2_images/turtle_2.png  
  inflating: /content/part2_images/fork_1.png  
 extracting: /content/part2_images/turtle_1.png  
 extracting: /content/part2_images/butterfly_2.png  
 extracting: /content/pa

## Helper Functions

## Code

In [ ]:
def initialize(im1, mean1, mean2, scale_fact_1_to_2, im2, rows, cols):
  '''
  Apply translation & scaling to im1 as a way of initialization
  '''
  # Translation
  old_img = copy.copy(im1)
  im1 = (mean2-mean1) + im1

  # Scaling
  new_mean = np.mean(im1, axis=0)
  im1 = im1 - new_mean
  im1 = im1 * scale_fact_1_to_2
  im1 = im1 + new_mean
  im1[:, 0] = np.clip(im1[:, 0], 0, rows-1)
  im1[:, 1] = np.clip(im1[:, 1], 0, cols-1)
  return im1

def find_NN(i, im2):
  distance = 1e8
  for j in im2:
    curr_distance = np.linalg.norm(i-j)
    if curr_distance < distance:
      distance = curr_distance
      nn = j
  return nn

def warp_image(im1, m1, m2, m3, m4, t1, t2, rows, cols):
  warped_im1 = []
  for i in im1:
    warped_im1.append([np.clip(m1*i[0]+m2*i[1]+t1, 0, rows-1), np.clip(m3*i[0]+m4*i[1]+t2, 0, cols-1)])
  return warped_im1

def align_shape(im1, im2, num_iter=50, show_every_iter=False):
  '''
  im1: input edge image 1
  im2: input edge image 2

  Output: transformation T [3] x [3]
  '''
  orig_im2 = copy.copy(im2)
  (rows, cols) = im1.shape
  idx_grid = np.mgrid[0:rows, 0:cols]
  idx_grid = np.swapaxes(idx_grid, 0, 1)
  idx_grid = np.swapaxes(idx_grid, 1, 2)
  im1 = idx_grid[im1==255] # Gives coordinates of pixels in (row, col) format
  im2 = idx_grid[im2==255]
  mean1 = np.mean(im1, axis=0)
  mean2 = np.mean(im2, axis=0)
  scale1 = np.std(im1, axis=0)
  scale2 = np.std(im2, axis=0)
  scale_fact_1_to_2 = scale2 / scale1
  im1 = initialize(im1, mean1, mean2, scale_fact_1_to_2, im2, rows, cols)

  for iter in range(num_iter):
    matches = []
    for i in im1:
      nn_of_i = find_NN(i, im2)
      matches.append([i, nn_of_i])
    A = []
    B = []
    for match in matches:
      x = match[0][0]
      y = match[0][1]
      x_dash = match[1][0]
      y_dash = match[1][1]
      A.append([x, y, 0, 0, 1, 0])
      A.append([0, 0, x, y, 0, 1])
      B.append(x_dash)
      B.append(y_dash)
    m1, m2, m3, m4, t1, t2 = np.linalg.lstsq(A, B, rcond=None)[0]
    im1 = warp_image(im1, m1, m2, m3, m4, t1, t2, rows, cols)
    if (show_every_iter):
      test = copy.copy(orig_im2)
      for i in range(len(im1)):
        test[int(im1[i][0]), int(im1[i][1])] = 255
      cv2_imshow(test)
  return im1

In [ ]:
def evalAlignment(aligned1, im2):
  '''
  Computes the error of the aligned image (aligned1) and im2, as the
  average of the average minimum distance of a point in aligned1 to a point in im2
  and the average minimum distance of a point in im2 to aligned1.
  '''
  d2 = ndimage.distance_transform_edt(1-im2) #distance transform
  err1 = np.mean(np.mean(d2[aligned1 > 0]))
  d1 = ndimage.distance_transform_edt(1-aligned1);
  err2 = np.mean(np.mean(d2[im2 > 0]))
  err = (err1+err2)/2;
  return err

def displayAlignment(im1, im2, aligned1, thick=False):
  '''
  Displays the alignment of im1 to im2
     im1: first input image to alignment algorithm (im1(y, x)=1 if (y, x) 
      is an original point in the first image)
     im2: second input image to alignment algorithm
     aligned1: new1(y, x) = 1 iff (y, x) is a rounded transformed point from the first time 
     thick: true if a line should be thickened for display
  ''' 
  if thick:
    # for thick lines (looks better for final display)
    dispim = np.concatenate((cv2.dilate(im1.astype('uint8'), np.ones((3,3), np.uint8), iterations=1), \
                             cv2.dilate(aligned1.astype('uint8'), np.ones((3,3), np.uint8), iterations=1), \
                             cv2.dilate(im2.astype('uint8'), np.ones((3,3), np.uint8), iterations=1)), axis=-1)
  else:
    # for thin lines (faster)
    dispim = np.concatenate((im1, aligned1, im2), axis = -1)
  return dispim
  

In [ ]:
imgPath = '/content/part2_images/';

objList = ['apple', 'bat', 'bell', 'bird', 'Bone', 'bottle', 'brick', \
    'butterfly', 'camel', 'car', 'carriage', 'cattle', 'cellular_phone', \
    'chicken', 'children', 'device7', 'dog', 'elephant', 'face', 'fork', 'hammer', \
    'Heart', 'horse', 'jar', 'turtle']

numObj = len(objList)

for idx in range(11, numObj):
  start_time = time.time()
  im1 = cv2.imread(imgPath+objList[idx]+'_1.png', 0)
  im2 = cv2.imread(imgPath+objList[idx]+'_2.png', 0)
  (rows, cols) = im1.shape

  aligned_im1_coords = align_shape(im1, im2)
  end_time = time.time()
  aligned_im1_coords = np.array(aligned_im1_coords)
  aligned_im1 = np.zeros((rows, cols))
  for i in range(np.shape(aligned_im1_coords)[0]):
    aligned_im1[int(aligned_im1_coords[i][0]), int(aligned_im1_coords[i][1])] = 255
  print('Runtime for ', objList[idx], '= ', end_time - start_time, ' seconds')
  error = evalAlignment(aligned_im1, im2)
  dispim = displayAlignment(im1, im2, aligned_im1, thick=True)
  print('Error for ', objList[idx], '= ', error)
  cv2_imshow(dispim)
  


## Write-up


#### 1) Explanation of Algorithm, initialization & Transformation model:

##### - Algorithm:
- Firstly, the edge points in both the images are calculated (by storing the coordinates of the pixels whose value is 255).
- Secondly, these arrays of edge points are used to generate an initial transformation as described in the next sub-section.
- Then, the Iterative Closest Point algorithm is applied in the following way:
  - Determine the match for a point in image1 by finding the point in image2 which has the smallest Euclidean distance. Repeat this for all the points in image1.
  - Get the transformation that satisfies these pairs of matches as close as possible as described in the 'Transformation model' sub-section below.
  - Using the estimated transformation matrix, warp all the points in image1.
  - Repeat the above steps feeding the output of one iteration as the input for the next iteration.
- The output of the final iteration gives the final version of image1 aligned with image2.

##### - Initialization:
  - The first image is translated & scaled in the following way to make it look closer to the second image.
  - First, the difference in the centroids between both the images is calculated & all the points in image1 are translated by this value. Let's call this image `translated_img`.
  - Then, the standard deviation of each image is calculated & the scale factor is defined as `std_dev of image2 / std_dev of image1`.
  - Finally, scaling is performed by radially stretching (from centroid) every point in `translated_img` by this `scale factor`.
  - <ins>Note</ins>: 'image' in this sub-section referes to the array of edge points in a picture.

##### - Transformation model:
  - An affine transformation model is used to fit the pairs of matches between both the images.
  - This is done using Direct Linear transformation.
  - A Least squares approach is used to find the parametres of the transformation.
  - <ins>Note</ins>: 'image' in this sub-section refers to the array of edge points in a picture.

#### 2) Results:
- IMAGE 1 ----------------------------------- TRANSFORMATION APPLIED ON IMAGE 1 ------------------------ IMAGE 2
- <img src="https://drive.google.com/uc?id=19IrJ3AsmIOTIHt-fVArm0QxqNOF6hlhc" align="center"/>

  - Runtime:194
  - Error:174
- <img src="https://drive.google.com/uc?id=106IkD29-eYkdbj-hKnsCFebqGlgKjwu-" align="center"/>

  - Runtime:1183
  - Error:425
- <img src="https://drive.google.com/uc?id=1M1tDu-_0yUv4-3E_GRUd8B44m4Nk8X8x" align="center"/>

  - Runtime:125
  - Error:179
- <img src="https://drive.google.com/uc?id=1HAGqeu1c-7OtRKgBff5ALAGFNLPmnwvi" align="center"/>

  - Runtime:469
  - Error:221
- <img src="https://drive.google.com/uc?id=1z9K5JhKtOJtCbEugL9OsHSfwaKfJrLCz" align="center"/>

  - Runtime:528
  - Error:320
- <img src="https://drive.google.com/uc?id=1Q-hvSrUirBOOO-2rlIG8eyIs2rCo8b3c" align="center"/>

  - Runtime:49
  - Error:246
- <img src="https://drive.google.com/uc?id=1BzhlGskNeLPRr-fS6u9PtwheGVIKjMDd" align="center"/>

  - Runtime:115
  - Error:303
- <img src="https://drive.google.com/uc?id=1Bu-wYZDRw_-Vg4fdV3wT1kKVZ_lGaBXc" align="center"/>

  - Runtime:464
  - Error:212
- <img src="https://drive.google.com/uc?id=1aDSvtY_TfkCEgBJ1KqCMYyCR-adaUX8Y" align="center"/>

  - Runtime:868
  - Error:269
- <img src="https://drive.google.com/uc?id=1KPNNB3MMbnp9dn4ETaHhXgPsb7ewfcAp" align="center"/>

  - Runtime:91
  - Error:242
- <img src="https://drive.google.com/uc?id=1U0eas9COqcj2R7Zragppsb-BpSLdOrdz" align="center"/>

  - Runtime:211
  - Error:284
- <img src="https://drive.google.com/uc?id=1YzWMeo0QmWVkgCWzwWMU-L7pv06cFEm2" align="center"/>

  - Runtime:2742
  - Error:392
- <img src="https://drive.google.com/uc?id=1c8gtSRW2c6HF3D7-oiw5MZknJ18-kKDv" align="center"/>

  - Runtime:360
  - Error:258
- <img src="https://drive.google.com/uc?id=1FbAgczb6bzJ-nuzvC1xIRWBSNrYnjHZO" align="center"/>

  - Runtime:237
  - Error:144
- <img src="https://drive.google.com/uc?id=1T4w99rLLFI-e37_lseIOga-9zWuuzEiX" align="center"/>

  - Runtime:63
  - Error:155
- <img src="https://drive.google.com/uc?id=1aUcR-J3ViBLqDU2QBjBI7_rwqex4FHzh" align="center"/>

  - Runtime:5258
  - Error:382
- <img src="https://drive.google.com/uc?id=1Jyw2tEcqXja4YiWf6-7qV0vWRS9xp2-f" align="center"/>

  - Runtime:2141
  - Error:351
- <img src="https://drive.google.com/uc?id=1V3GtwterQUnx1wcCqVizxdCObSQgsPrQ" align="center"/>

  - Runtime:4196
  - Error:562
- <img src="https://drive.google.com/uc?id=1gpmsr3IOw91QtGz4u-ueMLH_YXwXUutm" align="center"/>

  - Runtime:222
  - Error:232
- <img src="https://drive.google.com/uc?id=1wHkWNe9WuflBibGXfP-PrgZyCS9NhUY_" align="center"/>

  - Runtime:1409
  - Error:357
- <img src="https://drive.google.com/uc?id=1V4lUww35Z6kerzIcy2JsJJLb0_JZ8Cbp" align="center"/>

  - Runtime:151
  - Error:156
- <img src="https://drive.google.com/uc?id=1l6YYA8EeLd2pPNLFM_Gfgi6qs5rW0rQp" align="center"/>

  - Runtime:677
  - Error:311
- <img src="https://drive.google.com/uc?id=1bYlV3GaT-yhm_kB9gMM0vPKaS30mHlvb" align="center"/>

  - Runtime:4387
  - Error:492
- <img src="https://drive.google.com/uc?id=1pXpbBho3Oq9pKGPypzebZVXechqKUNTa" align="center"/>

  - Runtime:1463
  - Error:386
- <img src="https://drive.google.com/uc?id=1hSmUT9s_iOAknjDx7peMoZK4vl4iqwQE" align="center"/>

  - Runtime:400
  - Error:250

<ins>Note</ins>: The above runtimes are in seconds.